In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from config import get_cfg
from op import Operation

cfg = get_cfg()

operation = Operation(cfg)

choices=[
        'All','Encoder','Decoder','Center1','Center2',
        'Mid1','Mid1-Encoder','Mid1-Decoder',
        'Mid2','Mid2-Encoder','Mid2-Decoder',
        'Classifier'
]
paths=[
    'D-All-brains18=0409-151829','D-Encoder-brains18=0409-151856','D-Decoder-brains18=0409-151934',
    'D-Center1-brains18=0409-152036','D-Center2-brains18=0409-152150',
    
    'D-Mid1-brains18=0409-133726',
    'D-Mid1-Encoder-brains18=0409-133812',
    'D-Mid1-Decoder-brains18=0409-133906',
    
    'D-Mid2-brains18=0410-013143',
    'D-Mid2-Encoder-brains18=0410-013241',
    'D-Mid2-Decoder-brains18=0410-013259',
    
    'D-Classifier-brains18=0409-152309',
]

In [ ]:
operation.rm_dir()

In [ ]:
operation.cfg.model_type = 'Mid2-Encoder'
operation.load(path='./exp/D-Mid2-Encoder-brains18=0410-013241/model.pt')

In [ ]:
def myeval():
    operation.ensemble_load_models()
    operation.cfg.log.critical("Ensemble")

    # 评价val，采样
    operation.cfg.log.critical("评价val，采样")
    dice1, dice2 = operation.eval_sample_model_dices(operation.val_data, False)

    # 评价train，采样
    operation.cfg.log.critical("评价train，采样")
    dice1, dice2 = operation.eval_sample_model_dices(operation.train_data, False)
myeval()

In [ ]:
def myeval(choices, paths):
    operation.cfg.log.critical(choices)
    operation.cfg.log.critical(paths)
    for i, t in enumerate(choices):
        operation.cfg.model_type = t
        operation.load(path='./exp/{}/model.pt'.format(paths[i]))
        operation.cfg.log.critical(operation.cfg.model_type)
        
        # 用data评价模型，不采样
        operation.cfg.log.critical("用data评价模型，不采样")
        dice1, dice2 = operation.eval_model_dices(operation.val_data)
        dice1, dice2 = operation.eval_model_dices(operation.train_data)

        # 评价val，采样
        operation.cfg.log.critical("评价val，采样")
        dice1, dice2 = operation.eval_sample_model_dices(operation.val_data, True)

        # 评价train，采样
        operation.cfg.log.critical("评价train，采样")
        dice1, dice2 = operation.eval_sample_model_dices(operation.train_data, True)

myeval(choices=['No'], paths=['D-No-brains18=0409-165939'])

In [ ]:
# 用loader评价模型，不采样
dices, loss = operation.eval_model(operation.val_loader)
print(dices[1:].mean())
dices, loss = operation.eval_model(operation.train_loader)
print(dices[1:].mean())

In [ ]:
# 用data评价模型，不采样
dice1, dice2 = operation.eval_model_dices(operation.val_data)
dice1, dice2 = operation.eval_model_dices(operation.train_data)

In [ ]:
# 评价val，采样
# operation.cfg.sample_T = 30
dice1, dice2 = operation.eval_sample_model_dices(operation.val_data, True)

In [ ]:
# 评价train，采样
dice1, dice2 = operation.eval_sample_model_dices(operation.train_data, True)

In [ ]:
# 普通预测，不采样
def show_result(data, ids):
    for i in ids:
        imgs, folder, slice_id = data[i]
        operation.predict(imgs[2], imgs[3], title='{} {}'.format(folder, slice_id))
show_result(operation.val_data, [1])

In [ ]:
# 加噪声或异物，采样
import numpy as np
# noise = np.random.normal(loc=0.0, scale=0, size=(1, 240,240)).astype(np.float32)
noise = np.zeros(shape=(1, 240, 240), dtype=np.float32)
noise[0][20:40, 20:40] += 5
noise[0][80:100, 120:140] += 0
print(noise.shape, noise.dtype)
def show_result(data, ids):
    for i in ids:
        imgs, folder, slice_id = data[i]
        operation.predict_sample(imgs[2] + noise, imgs[3], title='{} {}'.format(folder, slice_id), sample=False)
show_result(operation.val_data, [20])

In [ ]:
# 加载肿瘤val集
operation.load_val_data(is_tumor=True)

In [ ]:
operation.ensemble_load_models()

In [ ]:
def show_result(data, ids):
    for i in ids:
        imgs, folder, slice_id = data[i]
        operation.predict_sample(imgs[2], imgs[3], title='{} {}'.format(folder, slice_id), sample=False)

# show_result(operation.val_data, [0])
# show_result(operation.val_data, [8])
# show_result(operation.val_data, [20])
# show_result(operation.val_data, [26])
# show_result(operation.val_data, [37])
# show_result(operation.val_data, [42])
# show_result(operation.val_data, [47])

show_result(operation.val_data, [20])
show_result(operation.val_data, [22])
show_result(operation.val_data, [24])
show_result(operation.val_data, [26])


In [ ]:
# 采样
# operation.cfg.sample_T = 30
def show_result(data, ids):
    for i in ids:
        imgs, folder, slice_id = data[i]
        operation.predict_sample(imgs[2], imgs[3], title='{} {}'.format(folder, slice_id), sample=True)


show_result(operation.val_data, [20])
show_result(operation.val_data, [22])
show_result(operation.val_data, [24])
show_result(operation.val_data, [26])





In [ ]:
# 删除文件夹
operation.rm_dir()

In [ ]:
operation.fit()

In [ ]:
from unet import UNet, DropoutUNet
from torchsummary import summary
model = DropoutUNet(n_channels=1, n_classes=9, model_type='All', drop_rate=0.2).cuda()
summary(model, (1, 240, 240))